In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re 

In [2]:
def notAPhoto(x):
    return "upload" not in x[2]

def getWikiLink(cell):
    allLinks = list(cell.iterlinks())
    goodLinks = list(filter(lambda x: notAPhoto(x), allLinks))
    if len(goodLinks)==0:
        return "ERROR: No Link"
    else:
        return goodLinks[0][2].replace('/wiki/','')

def getWikiString(string):
    return string.replace('https://en.wikipedia.org/wiki/','')

def getWikiDetails(cells):
    player = cells[0]
    country_cell = cells[1]
    display_name = player.text_content()
    return display_name, getWikiLink(player), getWikiLink(country_cell)

def getFullWikiLink(cell):
    allLinks = list(cell.iterlinks())
    goodLinks = list(filter(lambda x: notAPhoto(x), allLinks))
    if len(goodLinks)==0:
        return "ERROR: No Link"
    else:
        return goodLinks[0][2].replace('/wiki/','https://en.wikipedia.org/wiki/')

def getSquadUrl(url):
    return 'https://en.wikipedia.org/wiki/2019-20_'+ url +'_season'

In [29]:
def shouldSkip(text):
    return text.isnumeric() or len(text) == 0 or text == 'Name' or text == 'Goals' or text == 'Player'

def ifFive(row):
        print(row.text_content())
        return true
        
# def ifNormal(row):
    

def validPlayerRow(row):
    skip = False
    goodLength = len(row) in [5,10,12,13,14]
    if goodLength in [10,12,13,14]:
        name_cell = row[3]
        nationality_cell = row[2]
        text = name_cell.text_content().strip()
        skip = shouldSkip(text)
    if goodLength in [5]:
        skip = ifFive(row)
    return goodLength and not skip

def validPlayersTable(table):
    content = table.text_content()
    appsAndGoals = "App" in content and "Goal" in content
    playerOrName = "Player" in content or "Name" in content
    return appsAndGoals and playerOrName

def getPlayers(league,country,teamName,url):
    playerRows = []
    playerWikis = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    allTables = list(doc.xpath('//table'))
    goodTables = list(filter(lambda x: validPlayersTable(x), allTables))
    print("\n",teamName, url)
    if len(goodTables) > 0:
        playerTable = goodTables[0]
        headers = list(playerTable.xpath('//tr'))
        for header in headers:
            print('\t\t NEW PLAYER TABLE --------------')
            print(header.text_content())
#         goodRows = list(filter(lambda x: validPlayerRow(x), allRows))
#         for row in goodRows:
#             playerCell = row[3]
#             playerWiki = getFullWikiLink(playerCell)
#             playerName = playerCell.text_content().strip()
#             if playerWiki in playerWikis:
#                 print('Skipping')
#             else:
#                 print(playerName)
    else:
        print('Fucked it for: ',teamName,url)


In [30]:
getPlayers('Prem','England','Chelsea','https://en.wikipedia.org/wiki/2019-20_Chelsea_F.C._season')


 Chelsea https://en.wikipedia.org/wiki/2019-20_Chelsea_F.C._season
		 NEW PLAYER TABLE --------------
2019–20 season
		 NEW PLAYER TABLE --------------
Owner
		 NEW PLAYER TABLE --------------
Chairman
		 NEW PLAYER TABLE --------------
Head coach
		 NEW PLAYER TABLE --------------
Stadium
		 NEW PLAYER TABLE --------------
Premier League
		 NEW PLAYER TABLE --------------
FA Cup
		 NEW PLAYER TABLE --------------
EFL Cup
		 NEW PLAYER TABLE --------------
UEFA Champions League
		 NEW PLAYER TABLE --------------
UEFA Super Cup
		 NEW PLAYER TABLE --------------
Top goalscorer
		 NEW PLAYER TABLE --------------
Highest home attendance
		 NEW PLAYER TABLE --------------
Lowest home attendance
		 NEW PLAYER TABLE --------------
Average home league attendance
		 NEW PLAYER TABLE --------------
Biggest win
		 NEW PLAYER TABLE --------------
Biggest defeat
		 NEW PLAYER TABLE --------------
Position

		 NEW PLAYER TABLE --------------
Name

		 NEW PLAYER TABLE --------------
No.

		 NEW PLA

In [17]:
def getTeams(url,league,country):
#     outputData = pd.DataFrame([], columns = ['team','name','country','apps', 'wiki'])
    outputData = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 4:
            cell = row[0]
            teamName = cell.text_content().strip()
            wiki = getWikiLink(cell)
            skip = teamName == 'Team' or teamName == 'Rank' or teamName.isnumeric() or "ERROR" in wiki
            if not skip:
                getPlayers(league,country,teamName,getSquadUrl(wiki))

In [18]:
teams = getTeams('https://en.wikipedia.org/wiki/2019%E2%80%9320_Premier_League','Premier League','England')


 Arsenal https://en.wikipedia.org/wiki/2019-20_Arsenal_F.C._season
Bernd Leno
Héctor Bellerín
Kieran Tierney
Mohamed Elneny
Sokratis Papastathopoulos
Henrikh Mkhitaryan
Dani Ceballos
Alexandre Lacazette
Mesut Özil
Lucas Torreira
Pierre-Emerick Aubameyang
Ainsley Maitland-Niles
Rob Holding
Cédric Soares
Nicolas Pépé
Shkodran Mustafi
Calum Chambers
Pablo  Marí
David Luiz
Reiss Nelson
Emiliano Martínez
Konstantinos Mavropanos
Joe Willock
Matteo Guendouzi
Eddie Nketiah
Sead Kolašinac
Emile Smith Rowe
Matt Macey
Granit Xhaka
Gabriel Martinelli
Bukayo Saka
William Saliba
Bernd Leno
Héctor Bellerín
Kieran Tierney
Sokratis Papastathopoulos
Dani Ceballos
Alexandre Lacazette
Mesut Özil
Lucas Torreira
Pierre-Emerick Aubameyang
Ainsley Maitland-Niles
Rob Holding
Cédric
Nicolas Pépé
Shkodran Mustafi
Calum Chambers
Pablo Marí
David Luiz
Reiss Nelson
Emiliano Martínez
Joe Willock
Matteo Guendouzi
Eddie Nketiah
Sead Kolašinac
Matt Macey
Granit Xhaka
Gabriel Martinelli
Bukayo Saka
Henrikh Mkhitaryan
K